In [1]:
from ludwig.api import LudwigModel
import pandas as pd

/home/ankdesh/explore/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ankdesh/explore/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
/home/ankdesh/explore/.venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was

/home/ankdesh/explore/.venv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


/home/ankdesh/explore/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
df = pd.read_csv('/home/ankdesh/temp/rotten_tomatoes.csv')

In [4]:
print (df.dtypes)

Unnamed: 0          int64
movie_title        object
content_rating     object
genres             object
runtime           float64
top_critic           bool
review_content     object
recommended         int64
dtype: object


In [5]:
def process_column(col) -> None:
    print (col.name, col.dtypes)

In [6]:
def is_categorical (col, threshold_entries=10):
        if df[col].nunique() <= threshold_entries:
            return True
        else:
            return False 

In [7]:
def predict_ludwig_type(col)->str:
    if pd.api.types.is_integer_dtype(col):
        if is_categorical (col):
            return "category"
        else: 
            return "number"
    
    elif pd.api.types.is_float_dtype(col):
        return "number"
    
    elif pd.api.types.is_bool_dtype(col):
        return "binary"
    
    elif pd.api.types.is_datetime64_any_dtype(col):
        return "date"
        
    elif pd.api.types.is_object_dtype(col):
        return "text"

In [9]:
import pandas as pd

def infer_ludwig_dtype(column, categorical_threshold=3):
    """
    Infer the Ludwig data type for a Pandas DataFrame column.

    Parameters:
    column (pd.Series): The column to infer the data type for.
    categorical_threshold (float): The maximum number of unique values to consider a column as categorical.

    Returns:
    str: The inferred Ludwig data type.
    """
    if pd.api.types.is_bool_dtype(column):
        return 'binary'
    elif pd.api.types.is_integer_dtype(column):
        # Check if integer column should be treated as categorical
        if column.nunique() <= categorical_threshold:
            return 'category'
        else:
            return 'numerical'
    elif pd.api.types.is_float_dtype(column):
        return 'numerical'
    elif isinstance(column.dtype, pd.CategoricalDtype):
        return 'category'
    elif pd.api.types.is_object_dtype(column):
        # Further check if it is text or category based on unique values
        if column.nunique() <= categorical_threshold:
            return 'category'
        else:
            return 'text'
    elif pd.api.types.is_string_dtype(column):
        return 'text'
    elif pd.api.types.is_datetime64_any_dtype(column):
        return 'date'
    else:
        # Default to text if the type is not recognized
        return 'text'



In [10]:
# Sample usage
data = {
    'integer': [1, 2, 3, 4, 5, 6],
    'float': [1.1, 2.2, 3.3, 4.4, 5.5, 6.6],
    'boolean': [True, False, True, False, True, False],
    'string': ['text', 'more text', 'another text', 'text again', 'more text', 'another text'],
    'datetime': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06']),
    'category': pd.Series(['a', 'b', 'a', 'b', 'a', 'b'], dtype='category'),
    'int_category': [1, 1, 2, 2, 3, 3]  # Integer column that should be categorical
}

df = pd.DataFrame(data)

# Test the function
for column in df.columns:
    print(f"Column '{column}' is inferred as Ludwig datatype '{infer_ludwig_dtype(df[column])}'")

Column 'integer' is inferred as Ludwig datatype 'numerical'
Column 'float' is inferred as Ludwig datatype 'numerical'
Column 'boolean' is inferred as Ludwig datatype 'binary'
Column 'string' is inferred as Ludwig datatype 'text'
Column 'datetime' is inferred as Ludwig datatype 'date'
Column 'category' is inferred as Ludwig datatype 'category'
Column 'int_category' is inferred as Ludwig datatype 'category'


In [11]:
import pandas as pd

def infer_ludwig_dtype(column, categorical_threshold=0.1):
    """
    Infer the Ludwig data type for a Pandas DataFrame column.

    Parameters:
    column (pd.Series): The column to infer the data type for.
    categorical_threshold (float): The maximum proportion of unique values to consider a column as categorical.

    Returns:
    str: The inferred Ludwig data type.
    """
    if pd.api.types.is_bool_dtype(column):
        return 'binary'
    elif pd.api.types.is_integer_dtype(column):
        # Check if integer column should be treated as categorical
        if column.nunique() / column.size <= categorical_threshold:
            return 'category'
        else:
            return 'numerical'
    elif pd.api.types.is_float_dtype(column):
        return 'numerical'
    elif pd.api.types.is_categorical_dtype(column):
        return 'category'
    elif pd.api.types.is_object_dtype(column):
        # Further check if it is text or category based on unique values
        if column.nunique() / column.size <= categorical_threshold:
            return 'category'
        else:
            return 'text'
    elif pd.api.types.is_string_dtype(column):
        return 'text'
    elif pd.api.types.is_datetime64_any_dtype(column):
        return 'date'
    else:
        # Default to text if the type is not recognized
        return 'text'

def generate_ludwig_config(df, target_column):
    """
    Generate the input field of a Ludwig configuration file based on the DataFrame's columns.

    Parameters:
    df (pd.DataFrame): The input DataFrame.
    target_column (str): The name of the target column.

    Returns:
    dict: The input field of the Ludwig configuration file.
    """
    input_features = []
    for column in df.columns:
        if column == target_column:
            continue
        ludwig_type = infer_ludwig_dtype(df[column])
        input_features.append({
            'name': column,
            'type': ludwig_type
        })
    
    output_features = [{
        'name': target_column,
        'type': infer_ludwig_dtype(df[target_column])
    }]
    
    config = {
        'input_features': input_features,
        'output_features': output_features
    }
    return config

# Sample usage
data = {
    'integer': [1, 2, 3, 4, 5, 6],
    'float': [1.1, 2.2, 3.3, 4.4, 5.5, 6.6],
    'boolean': [True, False, True, False, True, False],
    'string': ['text', 'more text', 'another text', 'text again', 'more text', 'another text'],
    'datetime': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05', '2023-01-06']),
    'category': pd.Series(['a', 'b', 'a', 'b', 'a', 'b'], dtype='category'),
    'int_category': [1, 1, 2, 2, 3, 3],  # Integer column that should be categorical
    'target': [0, 1, 0, 1, 0, 1]  # Target column for the output feature
}



In [16]:
df = pd.read_csv('/home/ankdesh/temp/rotten_tomatoes.csv')
# Generate Ludwig config
config = generate_ludwig_config(df, 'recommended')
config


/tmp/ipykernel_76040/2011940009.py:24: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  elif pd.api.types.is_categorical_dtype(column):


{'input_features': [{'name': 'Unnamed: 0', 'type': 'numerical'},
  {'name': 'movie_title', 'type': 'text'},
  {'name': 'content_rating', 'type': 'category'},
  {'name': 'genres', 'type': 'category'},
  {'name': 'runtime', 'type': 'numerical'},
  {'name': 'top_critic', 'type': 'binary'},
  {'name': 'review_content', 'type': 'text'}],
 'output_features': [{'name': 'recommended', 'type': 'category'}]}

In [17]:
import yaml

In [18]:
yaml.dump(config,open("gen_config.yaml", "w"))

In [19]:
model = LudwigModel(config='gen_config.yaml')

In [20]:
results = model.train(dataset=df)

/home/ankdesh/explore/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:306: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1031.)
  return F.conv1d(input, weight, bias, self.stride,


In [23]:
results.train_stats.training

{'recommended': {'accuracy': [0.706938624382019,
   0.8636952638626099,
   0.932355523109436,
   0.9702885150909424,
   0.9878581166267395,
   0.9909813404083252,
   0.9931337833404541],
  'accuracy_micro': [0.7391071319580078,
   0.8750595450401306,
   0.93729168176651,
   0.9727380871772766,
   0.9888392686843872,
   0.9916964173316956,
   0.9936309456825256],
  'loss': [0.55918949842453,
   0.29113084077835083,
   0.15384788811206818,
   0.07121182978153229,
   0.031082915142178535,
   0.023528484627604485,
   0.01919330656528473],
  'roc_auc': [0.7935038805007935,
   0.9439418315887451,
   0.9843692779541016,
   0.996536910533905,
   0.9993084669113159,
   0.9995951652526855,
   0.9997155666351318]},
 'combined': {'loss': [0.55918949842453,
   0.29113084077835083,
   0.15384788811206818,
   0.07121182978153229,
   0.031082915142178535,
   0.023528484627604485,
   0.01919330656528473]}}